In [1]:
import uproot
import numpy as np
import matplotlib.pyplot as plt
import awkward as ak
import vector

vector.register_awkward()

import numba as nb
import hist

In [2]:
dir="test_mva_training_photon_cleaning_no_pu_jet"

#file = uproot.open("/eos/user/n/nchernya/cH_ntuples/hadded/output_ch_2018.root")
#file_1 = uproot.open("/eos/user/t/tbevilac/H+c/flashgg/out_test/HpC_4FS_flashgg.root")
#file = uproot.open("/eos/cms/store/cmst3/group/vhcc/hc/flashgg/20220520_cH_UL18/HPlusCharm_4FS_13TeV_125_tree.root")
#file = uproot.open("/eos/cms/store/cmst3/group/vhcc/hc/flashgg/20220520_cH_UL18/HPlusCharm_4FS_FXFX_13TeV_125_tree.root")
file = uproot.open("/eos/user/t/tbevilac/H+c/flashgg/out_test_020822/flashGG_4FS_fxfx.root")
#file = uproot.open("/eos/cms/store/cmst3/group/vhcc/hc/flashgg/20220520_cH_UL18/EGamma_2018A_tree.root")
#file = uproot.open("/eos/cms/store/cmst3/group/vhcc/hc/flashgg/20220520_cH_UL18/ggh2018_13TeV_125_tree.root")
file_data = uproot.open("/eos/cms/store/cmst3/group/vhcc/hc/flashgg/20220520_cH_UL18/EGamma_2018A_tree.root")
#file_ggh_1 = uproot.open("/eos/user/t/tbevilac/ggh_nanoaod_1.root")
#events      = ak.from_parquet("../H_DNA/test_cH_2018_complete_cleaning/merged_nominal.parquet")
#events      = ak.from_parquet("../H_DNA/test_cH_2018_cleaning_no_syst/merged_nominal.parquet")
#events      = ak.from_parquet("../H_DNA/test_Data_2018_photon_cleaning_new/merged_nominal.parquet")
#events      = ak.from_parquet("../H_DNA/test_mva_training_puId_tight/merged_nominal.parquet")
#events      = ak.from_parquet("../H_DNA/test_mva_training_jetId/merged_nominal.parquet")
#events      = ak.from_parquet("../../merged_nominal.parquet")
events      = ak.from_parquet("../H_DNA/"+dir+"/merged_nominal.parquet")

In [3]:
tree_data = file_data["tagsDumper/trees/Data_13TeV_cHTag_0;1"]
tree      = file["tagsDumper/trees/HPlusCharm_4FS_FXFX_13TeV_125_13TeV_cHTag_0;1"]

In [4]:
for branch in tree:
    print(branch)


<TBranch 'candidate_id' at 0x7fe56c92c310>
<TBranch 'weight' at 0x7fe56c92cb50>
<TBranch 'CMS_hgg_mass' at 0x7fe56c932340>
<TBranch 'sigmaMoM_decorr' at 0x7fe56c932af0>
<TBranch 'dZ' at 0x7fe56c9372e0>
<TBranch 'centralObjectWeight' at 0x7fe56c937a90>
<TBranch 'leadingJet_pflav' at 0x7fe56c93e280>
<TBranch 'leadingJet_hflav' at 0x7fe56c93ea30>
<TBranch 'leadingJet_nBHadrons' at 0x7fe56c943220>
<TBranch 'leadingJet_nCHadrons' at 0x7fe56c9439d0>
<TBranch 'leadCharmJet_CvsL_pflav' at 0x7fe56c94a1c0>
<TBranch 'leadCharmJet_CvsL_hflav' at 0x7fe56c94a970>
<TBranch 'leadCharmJet_CvsL_nBHadrons' at 0x7fe56c950160>
<TBranch 'leadCharmJet_CvsL_nCHadrons' at 0x7fe56c950910>
<TBranch 'leadCharmJet_CvsAll_pflav' at 0x7fe56c955100>
<TBranch 'leadCharmJet_CvsAll_hflav' at 0x7fe56c9558b0>
<TBranch 'leadCharmJet_CvsAll_nBHadrons' at 0x7fe56c955fd0>
<TBranch 'leadCharmJet_CvsAll_nCHadrons' at 0x7fe56c95a850>
<TBranch 'leadingJet_pt' at 0x7fe56c95af70>
<TBranch 'leadingJet_eta' at 0x7fe56c9617f0>
<TBranc

In [ ]:
events.fields

In [ ]:
#dZ   =tree["dZ"]
event   =tree["event"]
weight  =tree["weight"]
cOWeight=tree["centralObjectWeight"]
run     =tree["run"]
lumi    =tree["lumi"]
#sigmaRV =tree["sigmaRV"]
lPSigOverE=tree["leadingPhotonSigOverE"]
slPSigOverE=tree["subleadingPhotonSigOverE"]
sigma   =tree["sigmaMOverM"]
sigmadec=tree["sigmaMOverMDecorr"]
leadingJet_pt=tree["leadingJet_pt"]
leadingJet_eta=tree["leadingJet_eta"]
leadingJet_phi=tree["leadingJet_phi"]
leadingJet_mass=tree["leadingJet_mass"]
leadingPhoton_pt=tree["leadingPhoton_pt"]
leadingPhoton_eta=tree["leadingPhoton_eta"]
leadingPhoton_phi=tree["leadingPhoton_phi"]
subleadingPhoton_pt=tree["subleadingPhoton_pt"]
subleadingPhoton_eta=tree["subleadingPhoton_eta"]
subleadingPhoton_phi=tree["subleadingPhoton_phi"]
nJets=tree["nJets"]
diphoton_mass=tree["CMS_hgg_mass"]
LeadingPhotonIDMVA=tree["EGMLeadingPhotonIDMVA"]
SubLeadingPhotonIDMVA=tree["EGMSubLeadingPhotonIDMVA"]
leadingJet_puJetIdMVA=tree["leadingJet_puJetIdMVA"]
customLeadingPhotonIDMVA=tree["customLeadingPhotonIDMVA"]
customSubLeadingPhotonIDMVA=tree["customSubLeadingPhotonIDMVA"]
PhoJetMinDr=tree["PhoJetMinDr"]
nvtx=tree["nvtx"]
leadingJet_puJetIdMVA=tree["leadingJet_puJetIdMVA"]
PhoJetMinDr=tree["PhoJetMinDr"]
leadingJet_nBHadrons=tree["leadingJet_nBHadrons"]
CMS_hgg_mass=tree["CMS_hgg_mass"]

#dZ =dZ.array()
event =event.array()
weight =weight.array()
cOWeight = cOWeight.array()
run     =run.array()
lumi    =lumi.array()
sigma   =sigma.array()
sigmadec=sigmadec.array()

lPSigOverE=lPSigOverE.array()
slPSigOverE=slPSigOverE.array()
sigmaRV =0.5*np.sqrt(lPSigOverE**2 + slPSigOverE**2)
leadingJet_pt=leadingJet_pt.array()
leadingJet_eta=leadingJet_eta.array()
leadingJet_phi=leadingJet_phi.array()
leadingJet_mass=leadingJet_mass.array()
leadingPhoton_pt=leadingPhoton_pt.array()
leadingPhoton_eta=leadingPhoton_eta.array()
leadingPhoton_phi=leadingPhoton_phi.array()
subleadingPhoton_pt=subleadingPhoton_pt.array()
subleadingPhoton_eta=subleadingPhoton_eta.array()
subleadingPhoton_phi=subleadingPhoton_phi.array()
nJets=nJets.array()
diphoton_mass=diphoton_mass.array()
LeadingPhotonIDMVA=LeadingPhotonIDMVA.array()
SubLeadingPhotonIDMVA=SubLeadingPhotonIDMVA.array()
leadingJet_puJetIdMVA=leadingJet_puJetIdMVA.array()
customLeadingPhotonIDMVA=customLeadingPhotonIDMVA.array()
PhoJetMinDr=PhoJetMinDr.array()
customSubLeadingPhotonIDMVA=customSubLeadingPhotonIDMVA.array()
nvtx=nvtx.array()
leadingJet_puJetIdMVA=tree["leadingJet_puJetIdMVA"].array()
PhoJetMinDr=tree["PhoJetMinDr"].array()
leadingJet_nBHadrons=leadingJet_nBHadrons.array()
CMS_hgg_mass=CMS_hgg_mass.array()

In [ ]:
#dZ   =tree_data["dZ"]
event_data   =tree_data["event"]
weight_data  =tree_data["weight"]
#cOWeight=tree_data["centralObjectWeight"]
run_data     =tree_data["run"]
lumi_data    =tree_data["lumi"]
#sigmaRV =tree_data["sigmaRV"]
lPSigOverE_data=tree_data["leadingPhotonSigOverE"]
slPSigOverE_data=tree_data["subleadingPhotonSigOverE"]
sigma_data   =tree_data["sigmaMOverM"]
sigmadec_data=tree_data["sigmaMOverMDecorr"]
leadingJet_pt_data=tree_data["leadingJet_pt"]
leadingJet_eta_data=tree_data["leadingJet_eta"]
leadingJet_phi_data=tree_data["leadingJet_phi"]
leadingJet_mass_data=tree_data["leadingJet_mass"]
leadingPhoton_pt_data=tree_data["leadingPhoton_pt"]
leadingPhoton_eta_data=tree_data["leadingPhoton_eta"]
leadingPhoton_phi_data=tree_data["leadingPhoton_phi"]
subleadingPhoton_pt_data=tree_data["subleadingPhoton_pt"]
subleadingPhoton_eta_data=tree_data["subleadingPhoton_eta"]
subleadingPhoton_phi_data=tree_data["subleadingPhoton_phi"]
nJets_data=tree_data["nJets"]
diphoton_mass_data=tree_data["CMS_hgg_mass"]
LeadingPhotonIDMVA_data=tree_data["EGMLeadingPhotonIDMVA"]
SubLeadingPhotonIDMVA_data=tree_data["EGMSubLeadingPhotonIDMVA"]
leadingJet_puJetIdMVA_data=tree_data["leadingJet_puJetIdMVA"]
customLeadingPhotonIDMVA_data=tree_data["customLeadingPhotonIDMVA"]
customSubLeadingPhotonIDMVA_data=tree_data["customSubLeadingPhotonIDMVA"]
PhoJetMinDr_data=tree["PhoJetMinDr"]
nvtx_data=tree_data["nvtx"]
leadingJet_puJetIdMVA_data=tree_data["leadingJet_puJetIdMVA"]
PhoJetMinDr_data=tree_data["PhoJetMinDr"]
CMS_hgg_mass_data=tree_data["CMS_hgg_mass"]
#leadingJet_nBHadrons_data=tree_data["leadingJet_nBHadrons"]

#dZ =dZ.array()
event_data =event_data.array()
weight_data =weight_data.array()
#cOWeight = cOWeight.array()
run_data     =run_data.array()
lumi_data    =lumi_data.array()
sigma_data   =sigma_data.array()
sigmadec_data=sigmadec_data.array()

lPSigOverE_data=lPSigOverE_data.array()
slPSigOverE_data=slPSigOverE_data.array()
sigmaRV_data =0.5*np.sqrt(lPSigOverE_data**2 + slPSigOverE_data**2)
leadingJet_pt_data=leadingJet_pt_data.array()
leadingJet_eta_data=leadingJet_eta_data.array()
leadingJet_phi_data=leadingJet_phi_data.array()
leadingJet_mass_data=leadingJet_mass_data.array()
leadingPhoton_pt_data=leadingPhoton_pt_data.array()
leadingPhoton_eta_data=leadingPhoton_eta_data.array()
leadingPhoton_phi_data=leadingPhoton_phi_data.array()
subleadingPhoton_pt_data=subleadingPhoton_pt_data.array()
subleadingPhoton_eta_data=subleadingPhoton_eta_data.array()
subleadingPhoton_phi_data=subleadingPhoton_phi_data.array()
nJets_data=nJets_data.array()
diphoton_mass_data=diphoton_mass_data.array()
LeadingPhotonIDMVA_data=LeadingPhotonIDMVA_data.array()
SubLeadingPhotonIDMVA_data=SubLeadingPhotonIDMVA_data.array()
leadingJet_puJetIdMVA_data=leadingJet_puJetIdMVA_data.array()
customLeadingPhotonIDMVA_data=customLeadingPhotonIDMVA_data.array()
PhoJetMinDr_data=PhoJetMinDr_data.array()
customSubLeadingPhotonIDMVA_data=customSubLeadingPhotonIDMVA_data.array()
nvtx_data=nvtx_data.array()
leadingJet_puJetIdMVA_data=tree_data["leadingJet_puJetIdMVA"].array()
PhoJetMinDr_data=tree_data["PhoJetMinDr"].array()
#leadingJet_nBHadrons_data=leadingJet_nBHadrons_data.array()
CMS_hgg_mass_data=CMS_hgg_mass_data.array()